In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#used libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler as Scaler
from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [ ]:
#load data
data_set=pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")

# Exploring Data

In [ ]:
data_set.head()

In [ ]:
data_set.info()

In [ ]:
#rows x columns
data_set.shape

In [ ]:
#statistics of the data
data_set.describe()

In [ ]:
#check duplicates
data_set.duplicated().sum()

In [ ]:
#check null values
data_set.isnull().sum()

In [ ]:
#data distribution? 
data_set.hist(figsize = (20,20))

# Data Cleaning

In [ ]:
# Calculate the median value for BMI
median_BMI = data_set['BMI'].median()
print(median_BMI)
# Substitute it in the BMI column of the
# dataset where values are 0
data_set['BMI'] = data_set['BMI'].replace(
    to_replace=0, value=median_BMI)


In [ ]:
# Calculate the median value for BloodPressure
median_BD = data_set['BloodPressure'].median()
print(median_BD)
data_set['BloodPressure'] = data_set['BloodPressure'].replace(
    to_replace=0, value=median_BD)


In [ ]:
# Calculate the median value for Glucose
median_Glucose = data_set['Glucose'].median()
print(median_Glucose)
data_set['Glucose'] = data_set['Glucose'].replace(
    to_replace=0, value=median_Glucose)


In [ ]:
# Calculate the median value for SkinThickness
median_STH = data_set['SkinThickness'].median()
print(median_STH)
data_set['SkinThickness'] = data_set['SkinThickness'].replace(
    to_replace=0, value=median_STH)


In [ ]:
# Calculate the median value for Insulin
median_Insulin = data_set['Insulin'].median()
print(median_Insulin)
data_set['Insulin'] = data_set['Insulin'].replace(
    to_replace=0, value=median_Insulin)


In [ ]:
#how many women with 0 Pregnancies?
(data_set["Pregnancies"]==0).sum()

In [ ]:
data_set.describe()

# data visualization

In [ ]:
data_set.hist(figsize = (20,20))

In [ ]:
OUTCOMES=pd.Series(data_set['Outcome'].value_counts(ascending = False))
OUTCOMES

In [ ]:
OUTCOMES.plot.pie()

In [ ]:
corr = data_set.corr()
corr

In [ ]:
plt.figure(figsize=(12,10))
p=sns.heatmap(corr, annot=True)


# Splitting Data & Feature Scaling

In [ ]:
#x-->input(features) 
x=data_set.iloc[:,:-1].values
#y-->output(has diab or not)
y=data_set.iloc[:,-1].values

In [ ]:
# Split the training dataset in 2/3 / 1/3
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=1/3,random_state=42)

In [ ]:
# Apply a scaler
scaler = Scaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

# train a KNN model

In [ ]:
KNN1=KNeighborsClassifier(n_neighbors=14)
KNN1.fit(x_train,y_train)

In [ ]:
#predict x_test
y_pred_KNN_1=KNN1.predict(x_test)

In [ ]:
#show result
cn1=confusion_matrix(y_test,y_pred_KNN_1)
cn1

In [ ]:
#training better KNN model
KNN2=KNeighborsClassifier(leaf_size=10, n_neighbors=25, p=1,weights='uniform',algorithm='auto')
KNN2.fit(x_train,y_train)
#predict & show result
y_pred_KNN_2=KNN2.predict(x_test)
cn=confusion_matrix(y_test,y_pred_KNN_2)
cn

In [ ]:
#visualize result
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred_KNN_2)
p = sns.heatmap(pd.DataFrame(cnf_matrix), annot=True,fmt='g')
plt.title('Confusion matrix')

In [ ]:
#classification_report
print(classification_report(y_test,y_pred_KNN_2))

# train a SVC model

In [ ]:
model_svc=SVC(C=10.0,kernel='rbf',gamma='auto')
model_svc.fit(x_train,y_train)
y_pred_SVC=model_svc.predict(x_test)
cn=confusion_matrix(y_test,y_pred_SVC)
cn

In [ ]:
cnf_matrix_2 = metrics.confusion_matrix(y_test, y_pred_SVC)
p = sns.heatmap(pd.DataFrame(cnf_matrix_2), annot=True,fmt='g')
plt.title('Confusion matrix')

In [ ]:
print(classification_report(y_test,y_pred_SVC))

# Test models

In [ ]:
#extrnal data
#SVC_model
new_df = pd.DataFrame([[6, 168, 72, 35, 0, 43.6, 0.627, 65]])
new_df_scaled = scaler.transform(new_df)
prediction_SVC_1 = model_svc.predict(new_df_scaled)
prediction_SVC_1

In [ ]:
#KNN_model
prediction_KNN_1 = KNN2.predict(new_df_scaled)
prediction_KNN_1

In [ ]:
#random row from data
#SVC_model
row_3 = pd.DataFrame([[1,89,66,23,94,28.1,0.167,21]])
row_3_scaled=scaler.transform(row_3)
prediction_SVC_2 = model_svc.predict(row_3_scaled)
prediction_SVC_2

In [ ]:
#KNN_model
prediction_KNN_2 = KNN2.predict(row_3_scaled)
prediction_KNN_2

# save models

In [ ]:
import pickle
Pkl_Filename = "KNN.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(KNN2, file)
# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:  
    Pickled_LR_Model = pickle.load(file)

Pickled_LR_Model.predict(x_test)

In [ ]:
Pkl_Filename_2 = "SVC.pkl"  

with open(Pkl_Filename_2, 'wb') as file:  
    pickle.dump(model_svc, file)
# Load the Model back from file
with open(Pkl_Filename_2, 'rb') as file:  
    Pickled_LR_Model_2 = pickle.load(file)

Pickled_LR_Model_2.predict(x_test)